# Permutation flowshop scheduling problem

In [99]:
import pandas as pd
import random
import copy
import math

## Simulated annealing

- function for drawing the initial solution

In [1]:
def random_shuffle(len_data):
    startList = list(range(1, len_data+1))
    initialSolution = startList.copy()
    random.shuffle(initialSolution)
    return initialSolution

- function summing up the time of performed tasks

In [101]:
def timeSum(data, tasksOrder):    
    columns = len(data[0])
    rows = len(data)

    #create an empty list (like in excels)
    listOfTasks=[[0]*columns for i in range(rows)]

    #complete the list with the first row
    for i in range(columns):
        listOfTasks[0][i] = data[tasksOrder[0]-1][i]+listOfTasks[0][i-1]

    #complete the list with the first column   
    for j in range(1, rows):
        listOfTasks[j][0] = data[tasksOrder[j]-1][0]+listOfTasks[j-1][0]

    #complete the rest of the list items
    for i in range(1, columns):
        for j in range(1, rows):
            listOfTasks[j][i] = data[tasksOrder[j]-1][i] + max(listOfTasks[j-1][i], listOfTasks[j][i-1])

    return listOfTasks[rows-1][columns-1]

- a function that finds a new solution depending on the selected neighborhood

In [102]:
def solution(initialSolution, newSolution, neighborhood, list):
    twoRandomList = random.sample(range(len(list)), 2)

    if (neighborhood == "SWAP"):
        newSolution[twoRandomList[0]] = initialSolution[twoRandomList[1]]
        newSolution[twoRandomList[1]] = initialSolution[twoRandomList[0]]
    elif (neighborhood == "INVERSION"):
        if(twoRandomList[0] < twoRandomList[1]):
            newSolution[twoRandomList[0] : twoRandomList[1] + 1] = reversed(newSolution[twoRandomList[0] : twoRandomList[1] + 1])
        else:
            newSolution[twoRandomList[1] : twoRandomList[0] + 1] = reversed(newSolution[twoRandomList[1] : twoRandomList[0] + 1])
    elif (neighborhood == "INSERTION"):
        if(twoRandomList[0] < twoRandomList[1]):
            newSolution.insert(twoRandomList[0]+1, newSolution.pop(twoRandomList[1]))
        else:
            newSolution.insert(twoRandomList[1]+1, newSolution.pop(twoRandomList[0]))
    
    return newSolution

SA algorithm, parameters:
- `data`
- `alpha` - affects the rate of temperature reduction
- `t` - initial temperature
- `trialsPerEpoch` - number of trials in a given temperature
- `tMin` - minimal temperature, when it is reached, the algorithm ends
- `neighborhood` - the type of neighborhood (swap/inversion/insertion) 

In [103]:
def simulatedAnnealing(data, alpha, t, trialsPerEpoch, tMin, neighborhood):
    list = data.values.tolist()
    initialSolution = random_shuffle(len(list))
    sum = 499999
    sumSum = 500000 #so that we can save the smallest sum that has rolled over while the algorithm is running
    sumList = []
    while t > tMin:
        trial = 0
        while trial < trialsPerEpoch:
            newSolution = copy.deepcopy(initialSolution)
            solution(initialSolution, newSolution, neighborhood, list)
            
            newSum = timeSum(list, newSolution)
            oldSum = timeSum(list, initialSolution)

            #checking whether the new solution is better than the current one
            if(newSum < oldSum):
                initialSolution = newSolution
                sum = newSum
            else:
                if(random.uniform(0, 1) < math.exp(-(newSum - oldSum)/t)):
                    initialSolution = newSolution
                    sum = newSum
            trial += 1
        t = t * alpha
        #checking if the new smallest sum is less than the "by now" smallest sum
        if(sumSum > sum):
            sumSum = sum
            sumList = initialSolution
    print("Sum:", sum)
    print("Order of tasks:", initialSolution)
    if(sumSum < sum):
        print("\nPossible lower sum:", sumSum)
        print("Order of tasks:", sumList)

In [104]:
data50 = pd.read_excel("Dane_PFSP_50_20.xlsx", index_col=0)
data100 = pd.read_excel("Dane_PFSP_100_10.xlsx", index_col=0)
data200 = pd.read_excel("Dane_PFSP_200_10.xlsx", index_col=0)

- example of operation

In [ ]:
data = [data50, data100, data200]

for i in range(0,len(data)):
    print("======== SET:", i+1, "========")
    n = 0
    while n < 4:
        print("\nREP:", n+1)    
        simulatedAnnealing(data[i], 0.5, 500, 500, 10, "SWAP")
        n += 1

## Hill climbing

HC algorithm, parameters:
- `data`
- `iterationsWithoutImprovement` - after reaching given number of interations without improvement the algoritms ends
- `neighborhood` - the type of neighborhood (swap/inversion/insertion)
- `iterations` - after reaching given number of iterations the algoritm ends

In [106]:
def hillClimbing(data, iterationsWithoutImprovement, neighborhood, iterations):
    list = data.values.tolist()
    initialSolution = random_shuffle(len(list))
    sum = 0
    iFlat = 0
    numberOfTrials = 0
    while iterationsWithoutImprovement != iFlat and numberOfTrials != iterations:
        newSolution = copy.deepcopy(initialSolution)
        solution(initialSolution, newSolution, neighborhood, list)
        
        newSum = timeSum(list, newSolution)
        oldSum = timeSum(list, initialSolution)
        if newSum >= oldSum:
            iFlat = iFlat+1
        else: 
            initialSolution = newSolution
            sum = newSum
            iFlat = 0
        numberOfTrials = numberOfTrials+1

    print("Sum:", sum)
    print("Order of tasks:", initialSolution)

- example of operation

In [ ]:
data = [data50, data100, data200]

for i in range(0,len(data)):
    print("======== SET:", i+1, "========")
    n = 0
    while n < 4:
        print("\nREP:", n+1)    
        hillClimbing(data[i], 5000, "SWAP", 20000)
        n += 1

## NEH

In [108]:
def timeSum2(data, tasksOrder):    
    columns = len(data[0])
    rows = len(tasksOrder)

    #create an empty list (like in excels)
    listOfTasks=[[0]*columns for i in range(rows)]

    #complete the list with the first row
    for i in range(columns):
        listOfTasks[0][i] = data[tasksOrder[0]-1][i]+listOfTasks[0][i-1]

    #complete the list with the first column   
    for j in range(1, rows):
        listOfTasks[j][0] = data[tasksOrder[j]-1][0]+listOfTasks[j-1][0]

    #complete the rest of the list items
    for i in range(1, columns):
        for j in range(1, rows):
            listOfTasks[j][i] = data[tasksOrder[j]-1][i] + max(listOfTasks[j-1][i], listOfTasks[j][i-1])

    return listOfTasks[rows-1][columns-1]

NEH algorithm, parameters:
- `data`

In [109]:
def neh(data):
    summ = [[i + 1] * 2 for i in range(len(data))]
    for a in range(len(data)):
        summ[a][1] = sum(data[a])
    summ.sort(key = lambda x:x[-1], reverse = True)
    if(timeSum2(data, [summ[0][0], summ[1][0]]) > timeSum2(data, [summ[1][0], summ[0][0]])):
        order = [summ[1][0], summ[0][0]]
    else:
        order = [summ[0][0], summ[1][0]]
    for k in range(2,len(summ)):
        newOrder = copy.deepcopy(order)
        newTime = 999999
        for i in range(len(order) + 1):
            newOrder.insert(i, summ[k][0])
            if(timeSum2(data, newOrder) < newTime):
                newTime = timeSum2(data, newOrder)
                order=copy.deepcopy(newOrder)
            newOrder.remove(newOrder[i])
    dl = timeSum2(data, order)
    print("OrderOfTasks:", order)
    print("Sum:", dl)

In [110]:
df50 = data50.values.tolist()
df100 = data100.values.tolist()
df200 = data200.values.tolist()

- example of operation

In [111]:
neh(df200)

OrderOfTasks: [76, 91, 117, 19, 8, 31, 46, 129, 115, 18, 165, 48, 10, 110, 118, 99, 60, 126, 37, 6, 164, 61, 82, 147, 176, 130, 149, 113, 62, 20, 190, 189, 14, 137, 2, 144, 148, 9, 28, 141, 15, 57, 199, 157, 169, 64, 74, 43, 98, 21, 163, 63, 174, 16, 77, 86, 11, 111, 145, 44, 102, 73, 75, 133, 71, 154, 53, 121, 155, 140, 13, 1, 17, 24, 139, 30, 127, 80, 32, 41, 92, 40, 22, 69, 180, 93, 47, 51, 94, 183, 191, 175, 72, 114, 59, 182, 188, 7, 107, 192, 153, 200, 120, 151, 167, 89, 36, 68, 104, 171, 134, 26, 150, 12, 124, 178, 135, 197, 49, 81, 85, 78, 4, 27, 108, 112, 42, 173, 162, 119, 38, 142, 193, 196, 132, 143, 83, 160, 123, 156, 96, 187, 172, 177, 181, 23, 25, 158, 56, 97, 29, 65, 105, 84, 5, 166, 45, 170, 88, 35, 136, 55, 3, 70, 50, 168, 184, 79, 87, 161, 131, 109, 66, 34, 185, 106, 116, 159, 138, 95, 125, 186, 58, 198, 152, 194, 33, 103, 100, 52, 39, 128, 101, 67, 122, 54, 195, 179, 146, 90]
Sum: 10720
